In [18]:
import pandas as pd
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Pipeline,
    PreTrainedModel,
    PreTrainedTokenizerBase,
    pipeline,
)
import os
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct")

In [19]:
os.listdir('../../data/training_data')

['D10001.csv',
 'D90000.csv',
 'D20002.csv',
 'training_data.csv',
 'D20003.csv',
 'D20001.csv',
 'D30004.csv',
 'D30001.csv',
 'D30002.csv',
 'D30003.csv',
 'clem_top_10_models_data_all_successful_episodes_no_preprocessing.csv',
 'D70005.csv',
 'D70004.csv',
 'D70001.csv',
 'D70003.csv',
 'D70002.csv',
 'clem_top_10_models_data_all_successful_episodes_individual_turns_adapted_imagegame.csv',
 '.ipynb_checkpoints',
 'clem_top_10_models_data_all_successful_episodes_individual_turns.csv',
 'D50001.csv',
 'D50003.csv',
 'D8000X.csv',
 'D50002.csv',
 'D50006.csv',
 'D50007.csv',
 'D50005.csv',
 'D50004.csv']

In [20]:
all_datasets = ['D90000.csv']

In [21]:
for dataset in all_datasets:
    path = os.path.join('../../data/training_data', dataset)
    df = pd.read_csv(path)
    df['chat'] = df.chat.map(lambda x: eval(x))
    df["text"] = df.chat.map(lambda x: tokenizer.apply_chat_template(conversation=x, tokenize=False))
    df.to_csv(path, index=False)

In [22]:
df.head()

,game,game_id,model,benchmark_version,experiment,episode,Aborted,Lose,Success,chat,target,player,text
0,referencegame,18,gpt-4-0125-preview-t0.0--gpt-4-0125-preview-t0.0,v1.5,4_shape_grids,episode_18,0,0,1,"[{'role': 'user', 'content': 'You are given th...",first,player 1,<|begin_of_text|><|start_header_id|>system<|en...
1,taboo,16,gpt-4-t0.0--gpt-3.5-turbo-t0.0,v0.9,0_high_en,episode_16,0,0,1,"[{'role': 'user', 'content': 'You are playing ...",minnesota,player 1,<|begin_of_text|><|start_header_id|>system<|en...
2,imagegame,16,gpt-4-t0.0--gpt-4-t0.0,v0.9,0_compact_grids,episode_16,0,0,1,"[{'role': 'user', 'content': 'Let us play a ga...",D ▢ ▢ ▢ D\nD D ▢ D D\nD ▢ D ▢ D\nD ▢ ▢ ▢ D\nD ...,player 1,<|begin_of_text|><|start_header_id|>system<|en...
3,imagegame,16,gpt-4-t0.0--gpt-4-t0.0,v0.9,0_compact_grids,episode_16,0,0,1,"[{'role': 'user', 'content': 'Let us play a ga...",D ▢ ▢ ▢ D\nD D ▢ D D\nD ▢ D ▢ D\nD ▢ ▢ ▢ D\nD ...,player 1,<|begin_of_text|><|start_header_id|>system<|en...
4,imagegame,16,gpt-4-t0.0--gpt-4-t0.0,v0.9,0_compact_grids,episode_16,0,0,1,"[{'role': 'user', 'content': 'Let us play a ga...",D ▢ ▢ ▢ D\nD D ▢ D D\nD ▢ D ▢ D\nD ▢ ▢ ▢ D\nD ...,player 1,<|begin_of_text|><|start_header_id|>system<|en...
